In [ ]:
from numpy import log, corrcoef, unique, isnan, asarray
from numpy.random import shuffle

In [ ]:
from pandas import read_csv

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import altair as alt

alt.renderers.enable("default", embed_options={"actions": False})
alt.data_transformers.enable("data_server")
alt.data_transformers.disable_max_rows()

In [ ]:
df = read_csv("../../data/FIA_multipleDroughts_forBill_NoDroughtThresh.csv")

In [ ]:
df.loc[isnan(df["BAMort1(m2/ha)"]), "BAMort1(m2/ha)"] = 0
df.loc[isnan(df["BAMort2(m2/ha)"]), "BAMort2(m2/ha)"] = 0
df.loc[isnan(df["BAMort3(m2/ha)"]), "BAMort3(m2/ha)"] = 0
df.loc[isnan(df["BAMort4(m2/ha)"]), "BAMort4(m2/ha)"] = 0

In [ ]:
df.loc[:, "Ratio2"] = df["BAMort2(m2/ha)"] / df["BA1(m2/ha)"]
df.loc[:, "Ratio3"] = df["BAMort3(m2/ha)"] / df["BA2(m2/ha)"]
df.loc[:, "LogRatio2"] = log(df["BAMort2(m2/ha)"] / df["BA1(m2/ha)"] + 0.001)
df.loc[:, "LogRatio3"] = log(df["BAMort3(m2/ha)"] / df["BA2(m2/ha)"] + 0.001)

In [ ]:
df_ca = df[df["STATECD"] == 6]

In [ ]:
alt.Chart(df_ca).mark_point(clip=True).encode(
    alt.X("BA1(m2/ha)", scale=alt.Scale(domain=(0, 50))),
    alt.Y("BAMort2(m2/ha)", scale=alt.Scale(domain=(0, 50))),
)

In [ ]:
df_ca["LogRatio2"].max()

In [ ]:
alt.Chart(df_ca).mark_bar().encode(
    alt.X("LogRatio2", bin=alt.Bin(maxbins=50)),
    y="count()",
)

In [ ]:
unique(df.FORTYPCD)

In [ ]:
alt.Chart(df[df.FORTYPCD == 185]).mark_point(clip=True).encode(
    alt.X("min_prec_spei2", scale=alt.Scale(domain=(-4, -0.5))),
    alt.Y("LogRatio2", scale=alt.Scale(domain=(-8, 1))),
)

In [ ]:
alt.Chart(df[df["min_prec_spei2"] < -2]).mark_point(clip=True).encode(
    alt.X("BA1(m2/ha)", scale=alt.Scale(domain=(-5, 150))),
    alt.Y("Ratio2", scale=alt.Scale(domain=(-0.01, 0.7))),
    alt.Color("min_prec_spei2", scale=alt.Scale(domain=(-5, 0))),
)

In [ ]:
alt.Chart(df[df["min_prec_spei2"] > -2]).mark_point(clip=True).encode(
    alt.X("BA1(m2/ha)", scale=alt.Scale(domain=(-5, 150))),
    alt.Y("Ratio2", scale=alt.Scale(domain=(-0.01, 0.7))),
    alt.Color("min_prec_spei2", scale=alt.Scale(domain=(-5, 0))),
)

In [ ]:
isnan(df["BA1(m2/ha)"]).sum()

In [ ]:
x = df["min_prec_spei2"][df.FORTYPCD == 185]
y = df["LogRatio2"][df.FORTYPCD == 185]
isnan = (x.isnull() | y.isnull()) | (y < -6)
x = x[~isnan]
y = y[~isnan]

In [ ]:
corrcoef(x, y)

In [ ]:
y[x < -2].mean()

In [ ]:
y[(x > -2) & (x < -1.8)].mean()

In [ ]:
y[(x > -1.8) & (x < -1.5)].mean()

In [ ]:
y[(x > -1.5) & (x < -1.2)].mean()

In [ ]:
y[((x > -1.2) & (x < 0))].mean()

In [ ]:
plt.hist(log(y[(x > -2) & (x < -1.8)].dropna()), bins=100)

In [ ]:
df_ca = df[df["STATECD"] == 6]
df_ca["R2"] = log(df_ca["BAMort2(m2/ha)"] / df_ca["BA1(m2/ha)"])

In [ ]:
import altair as alt

alt.renderers.enable("default", embed_options={"actions": False})
alt.data_transformers.enable("data_server")
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(df_ca).mark_point(clip=True).encode(
    alt.X("BA1(m2/ha)", scale=alt.Scale(domain=(0, 50))),
    alt.Y("BAMort2(m2/ha)", scale=alt.Scale(domain=(0, 50))),
)

In [ ]:
alt.Chart(df_ca).mark_point(clip=True).encode(
    alt.X("min_prec_spei2", scale=alt.Scale(domain=(-3, 0))),
    alt.Y("R2", scale=alt.Scale(domain=(-6, 3))),
)

In [ ]:
import gcsfs
import zarr

In [ ]:
gcs = gcsfs.GCSFileSystem(anon=True)
store = gcsfs.GCSMap(
    root="carbonplan-scratch/terraclimate-fia-cond.zarr", gcs=gcs, check=False
)
root = zarr.group(store=store)

In [ ]:
a = asarray(root["PDSI"])

In [ ]:
asarray(root["time"])

In [ ]:
len(asarray(root["lon"]))